# <center> Lecture11 : Evaluating Regression Models </center>  
 
## <center> Instructor: Dr. Hu Chuan-Peng </center> 

## 问题回顾

还记得我们上节课的思考题吗？

🤔贝叶斯回归模型与传统的线性回归模型得到的结论是否一致？

首先，让我们回顾上节课定义的三种线性模型，并且通过 PyMC 进行定义和拟合。

|模型|参数|解释|  
|-|-|-|  
|model 1|RT ~ Label|简单线性回归模型：自变量为两水平的离散变量|   
|model 2|RT ~ Label + Matching|多元回归模型：自变量为两水平的离散变量和多水平的离散变量|  
|model 3|RT ~ Label + Matching + Label:Matching|多元回归模型：自变量额外增加了两个自变量间的交互作用|

In [1]:
# 导入 pymc 模型包，和 arviz 等分析工具 
import pymc as pm
import arviz as az
import seaborn as sns
import scipy.stats as st
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import os

# 忽略不必要的警告
import warnings
warnings.filterwarnings("ignore")

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [ ]:
try:
  df_raw = pd.read_csv('/home/mw/input/bayes3797/Kolvoort_2020_HBM_Exp1_Clean.csv')
except:
  df_raw = pd.read_csv('/data/Kolvoort_2020_HBM_Exp1_Clean.csv')

df = df_raw.groupby(['Subject','Label', 'Matching'], as_index=False)['RT_sec'].mean()

# 将 Label 列的数字编码转为文字标签
df['Label'] = df['Label'].replace({1: 'Self', 2: 'Friend', 3: 'Stranger'})

df['Matching'] = df['Matching'].replace({'Matching': 'matching', 'Nonmatching': 'nonmatching'})

# 设置索引
df["index"] = range(len(df))
df = df.set_index("index")

# 将 Label 列转换为有序的分类变量
df['Label'] = pd.Categorical(df['Label'], categories=['Self', 'Friend', 'Stranger'], ordered=True)

# 将分类变量转换为哑变量
X1 = (df['Label'] == 'Friend').astype(int)
X2 = (df['Label'] == 'Stranger').astype(int)

# Matching 条件的哑变量
Matching = (df['Matching'] == 'matching').astype(int)  

# Friend 和 Matching 的交互
Interaction_1 = X1 * Matching  

# Stranger 和 Matching 的交互
Interaction_2 = X2 * Matching  

In [3]:
# 建立模型1
with pm.Model() as model1:
    # 定义先验分布参数
    beta_0 = pm.Normal('beta_0', mu=5, sigma=2)
    beta_1 = pm.Normal('beta_1', mu=0, sigma=1)
    beta_2 = pm.Normal('beta_2', mu=0, sigma=1)
    sigma = pm.Exponential('sigma', lam=0.3)
    
    # 线性模型表达式
    mu = beta_0 + beta_1 * X1 + beta_2 * X2
    
    # 观测数据的似然函数
    likelihood = pm.Normal('Y_obs', mu=mu, sigma=sigma, observed=df['RT_sec'])

# 建立模型2和模型3

with pm.Model() as model2:
    # 先验分布
    beta_0 = pm.Normal('beta_0', mu=5, sigma=2)  
    beta_1 = pm.Normal('beta_1', mu=0, sigma=1)  
    beta_2 = pm.Normal('beta_2', mu=0, sigma=1)  
    beta_3 = pm.Normal('beta_3', mu=0, sigma=1)  
    sigma = pm.Exponential('sigma', lam=0.3)  
    
    # 线性模型
    mu = beta_0 + beta_1 * X1 + beta_2 * X2 + beta_3 * Matching
    
    # 观测数据的似然函数
    likelihood = pm.Normal('Y_obs', mu=mu, sigma=sigma, observed=df['RT_sec'])

with pm.Model() as model3:
    # 定义先验分布
    beta_0 = pm.Normal('beta_0', mu=5, sigma=2)  
    beta_1 = pm.Normal('beta_1', mu=0, sigma=1) 
    beta_2 = pm.Normal('beta_2', mu=0, sigma=1)  
    beta_3 = pm.Normal('beta_3', mu=0, sigma=1)  
    beta_4 = pm.Normal('beta_4', mu=0, sigma=1) 
    beta_5 = pm.Normal('beta_5', mu=0, sigma=1)  
    sigma = pm.Exponential('sigma', lam=0.3)  
    
    # 线性模型
    mu = (beta_0 + beta_1 * X1 + beta_2 * X2 + beta_3 * Matching +
          beta_4 * Interaction_1 + beta_5 * Interaction_2)
    
    # 观测数据的似然函数
    likelihood = pm.Normal('Y_obs', mu=mu, sigma=sigma, observed=df['RT_sec'])


In [4]:
#===========================
#     注意！！！以下代码可能需要运行3分钟左右
#===========================

def run_model_sampling(save_name, model=None, draws=5000, tune=1000, chains=4, random_seed=84735):
    """
    运行模型采样，并在结果不存在时进行采样，存在时直接加载结果。

    Parameters:
    - save_name: 用于保存或加载结果的文件名（无扩展名）
    - model: pymc 模型
    - draws: 采样次数 (默认5000)
    - tune: 调整采样策略的次数 (默认1000)
    - chains: 链数 (默认4)
    - random_seed: 随机种子 (默认84735)

    Returns:
    - trace: 采样结果
    """
    
    # 检查是否存在保存的.nc文件
    nc_file = f"{save_name}.nc"
    if os.path.exists(nc_file):
        print(f"加载现有的采样结果：{nc_file}")
        # 如果文件存在，则加载采样结果
        trace = az.from_netcdf(nc_file)
    else:

        assert model is not None, "模型未定义，请先定义模型"

        print(f"没有找到现有的采样结果，正在执行采样：{save_name}")
        # 如果文件不存在，则进行采样计算
        with model:
            trace = pm.sample_prior_predictive(draws=draws, random_seed=random_seed)
            idata = pm.sample(draws=draws,                   # 使用mcmc方法进行采样，draws为采样次数
                              tune=tune,                    # tune为调整采样策略的次数
                              chains=chains,                # 链数
                              discard_tuned_samples=True,   # tune的结果将在采样结束后被丢弃
                              idata_kwargs={"log_likelihood": True},
                              random_seed=random_seed)      # 后验采样

            trace.extend(idata)
            # 进行后验预测并扩展推断数据
            pm.sample_posterior_predictive(trace, extend_inferencedata=True, random_seed=random_seed)
            
            # 保存结果到指定文件
        trace.to_netcdf(nc_file)
        
    return trace

# 运行所有三个模型
model1_trace = run_model_sampling("lec10_model1",model1)
model2_trace = run_model_sampling("lec10_model2",model2)
model3_trace = run_model_sampling("lec10_model3",model3)

加载现有的采样结果：lec10_model1.nc
加载现有的采样结果：lec10_model2.nc
加载现有的采样结果：lec10_model3.nc


#### 与传统线性回归做法的对比：

为检测模型是否合理，我们可以将贝叶斯模型与传统的线性回归模型进行对比。

我们将以 model3 为例，使用statsmodels库来构建一个传统的线性回归模型，并将其结果与贝叶斯模型的结果进行比较。

In [6]:
import statsmodels.api as sm

# 构建设计矩阵
X = sm.add_constant(
    pd.DataFrame({
        'X1': X1, 
        'X2': X2, 
        'Matching': Matching,
        'Interaction_1': Interaction_1,
        'Interaction_2': Interaction_2
    })
)
y = df['RT_sec']

# 传统线性回归模型
model0 = sm.OLS(y, X).fit()

# 打印回归结果
model0.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 RT_sec   R-squared:                       0.062
Model:                            OLS   Adj. R-squared:                  0.036
Method:                 Least Squares   F-statistic:                     2.362
Date:                Wed, 27 Nov 2024   Prob (F-statistic):             0.0418
Time:                        11:04:32   Log-Likelihood:                 115.01
No. Observations:                 186   AIC:                            -218.0
Df Residuals:                     180   BIC:                            -198.7
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.7277      0.024     30.570      0.000       0.681       0.775
X1                0.0488      0.034      1.449      0.149      -0.018       0.115
X2                0.0326      0.034      0.970      0.333      -0.034       0.099
Matching         -0.0501      0.034     -1.487      0.139      -0.116       0.016
Interaction_1     0.0305      0.048      0.640      0.523      -0.063       0.124
Interaction_2     0.0568      0.048      1.194      0.234      -0.037       0.151
==============================================================================
Omnibus:                        7.639   Durbin-Watson:                   0.534
Prob(Omnibus):                  0.022   Jarque-Bera (JB):                7.449
Skew:                          -0.477   Prob(JB):                       0.0241
Kurtosis:                       3.227   Cond. No.                         9.77
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [7]:
# 注意，可以通过 az.plot_bf() 函数来计算贝叶斯因子（Bayes Factor）。
az.summary(model3_trace, hdi_prob=0.95)

,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta_0,0.729,0.024,0.682,0.775,0.0,0.0,8434.0,10823.0,1.0
beta_1,0.047,0.034,-0.021,0.112,0.0,0.0,9077.0,11295.0,1.0
beta_2,0.032,0.034,-0.034,0.097,0.0,0.0,9429.0,11853.0,1.0
beta_3,-0.051,0.033,-0.117,0.012,0.0,0.0,8437.0,11481.0,1.0
beta_4,0.032,0.047,-0.061,0.122,0.0,0.0,9113.0,11619.0,1.0
beta_5,0.058,0.047,-0.034,0.150,0.0,0.0,9541.0,12573.0,1.0
sigma,0.133,0.007,0.120,0.147,0.0,0.0,13797.0,13194.0,1.0


**我们可以对比两个模型的结果**

- 贝叶斯回归提供了更多关于参数不确定性的细节，
- 而传统的 OLS 回归提供了点估计、标准误差、置信区间和 p 值，适合用于传统的显著性检验和模型评估。


| 参数               | 贝叶斯回归（Bayesian Regression）                                   | 传统线性回归（OLS Regression）  |
|------------------|---------------------------------------------------|---------------------------------|
| **$\beta_0$**        | Mean: 0.729<br>SD: 0.024<br>HDI: [0.682, 0.775]<br>BF10: 0.2        | Mean: 0.7277<br>SE: 0.024<br>95% CI: [0.681, 0.775]<br>$p = 0.000$ |
| **$\beta_1$**        | Mean: 0.047<br>SD: 0.034<br>HDI: [-0.021, 0.112]<br>BF10: 0.09      | Mean: 0.0488<br>SE: 0.034<br>95% CI: [-0.018, 0.115]<br>$p = 0.149$ |
| **$\beta_2$**        | Mean: 0.032<br>SD: 0.034<br>HDI: [-0.034, 0.097]<br>BF10: 0.05      | Mean: 0.0326<br>SE: 0.034<br>95% CI: [-0.034, 0.099]<br>$p = 0.333$ |
| **$\beta_3$**        | Mean: -0.051<br>SD: 0.033<br>HDI: [-0.117, 0.012]<br>BF10: 0.1      | Mean: -0.0501<br>SE: 0.034<br>95% CI: [-0.116, 0.016]<br>$p = 0.139$ |
| **$\beta_4$**        | Mean: 0.032<br>SD: 0.047<br>HDI: [-0.061, 0.122]<br>BF10: 0.06      | Mean: 0.0305<br>SE: 0.048<br>95% CI: [-0.063, 0.124]<br>$p = 0.523$ |
| **$\beta_5$**        | Mean: 0.058<br>SD: 0.047<br>HDI: [-0.034, 0.150]<br>BF10: 0.1       | Mean: 0.0568<br>SE: 0.048<br>95% CI: [-0.037, 0.151]<br>$p = 0.234$ |
| **$\sigma$**         | Mean: 0.133<br>SD: 0.007<br>HDI: [0.120, 0.147]                    | -                               |
| **模型R²**           | -                                                                 | R-squared: 0.062 |
| **模型调整R²**       | -                                                                 | Adjusted R-squared: 0.036 |
| **F-statistic**      | -                                                                 | F-statistic: 2.362<br>$p = 0.0418$ |
| **Log-Likelihood**   | -                                                                 | Log-Likelihood: 115.01 |
| **AIC**              | -                                                                 | AIC: -218.0 |
| **BIC**              | -                                                                 | BIC: -198.7 |


<div style="padding-bottom: 30px;"></div>

## 模型评估与比较 (Model Evaluation & Comparison)  

在模型评估中，**贝叶斯回归**和**传统线性回归**模型在参数估计上的结果非常接近。但是，它们的区别在于对参数的不确定性评估。
* 贝叶斯模型提供了明确的参数分布，包括 **均值**、**标准差** 和 **高密度区间**（HDI），这使得我们可以更清晰地了解参数估计的不确定性。
* 而传统线性回归则侧重于给出参数的点估计，并通过 **标准误差** 和 **95% 置信区间** 进行评估。虽然传统模型没有明确给出不确定性，但它通过 ***p 值***、**R²** 等统计量提供了其他重要信息。


**🤔思考**

**传统回归分析**提供了更多的 **模型评估指标📊**，如 **R²**、**调整后的 R²**、**F 统计量**、**对数似然**、**AIC** 和 **BIC** 等。这些指标不仅帮助我们评估模型的拟合优度，还能有效地比较不同模型的复杂性和预测效果。

在第十课中，我们已经探索了多个研究假设和模型。那么，**哪一个模型对于数据的预测效果最好**呢？我们又该如何通过这些评估指标来做出更有力的比较呢？

**📈💡接下来的步骤：**
我们将深入讨论如何使用具体的 **评估指标** 来量化模型性能，并根据这些指标对模型进行**评估**和**比较**。

![Image Name](https://cdn.kesci.com/upload/image/rkz1dqyo2e.png?imageView2/0/w/700)

<div style="padding-bottom: 30px;"></div>

#### 什么是模型评估？


模型评估则是指对模型是否公平性、有效性、可信性进行评估，既可以是对单个模型，也可以是对多个模型进行。 

模型评估与比较(Model evaluation & comparison)的目的在于选择最好的模型。  

什么需要模型比较？  

1. 例如，比较 mode3 和 model2 可以帮助我们确定“Label”和“Matching”之间的交互或调节作用。  
2. 例如，比较 model2 和 model1 可以衡量增加预测因子是否能提升模型的预测能力。  
- 总之，模型比较的目的随着研究目的变化而变化。  

|模型|参数|解释|  
|-|-|-|  
|model 1|RT ~ Label|简单线性回归模型：自变量为两水平的离散变量|   
|model 2|RT ~ Label + Matching|多元回归模型：自变量为两水平的离散变量和多水平的离散变量|  
|model 3|RT ~ Label + Matching + Label:Matching|多元回归模型：自变量额外增加了两个自变量间的交互作用|




* 我们可以从以下三个角度来思考这个问题：  

1. 模型本身公正吗？(How fair is the model?)  
    - 公平性(How fair)：模型在数据收集和分析的整个流程中的公正性。  

2. 模型存在错误吗？(How wrong is the model?)  
   - 错误程度(How wrong)：模型在**实践**中是否有效？即是否能够准确地预测**样本**数据。
  
3.  后验预测模型有多准确？(How accurate are the posterior predictive models?) 
    - 准确性(How accurate)：模型是否反映**现实规律**？即是否能够准确地预测**样本外**数据。  




## 模型公正吗(Is the model fair?)  

模型公正性是一个上位概念，它描述了模型是否符合我们(社会、道德、伦理)的预期，而不仅是关注模型和样本数据的关系。  

我们可以借助几个相关问题来理解和思考模型的公众性：  

1. 数据的收集过程是怎样的？  

   - 数据的收集过程直接影响模型的公正性。如果数据收集的过程存在偏见或不充分考虑多样性，那么模型就可能会产生不公正的结果。

   - 例如，某些群体的数据可能被忽视或代表性不足，导致模型结果不具备广泛的适用性或公平性。

2. 数据由谁收集，数据收集的目的是什么？  
   
   - 数据收集者的身份、意图和研究目的，都会影响数据的性质和使用方式。
  
   - 资本主义的核心观念推动了个体主义和竞争性思维，可能导致研究样本的偏倚，削弱了全球不同文化和社会背景的代表性。
  


**1. 数据的收集的过程公平吗？**  

在本示例研究中，数据来源于基于自我匹配范式的实验数据，这些数据通过线下认知实验收集而得。

* 数据收集过程是公平的，被试填写了实验的知情同意书，并获得相应的报酬。  
* 此外，数据收集过程是匿名化的，保护了被试的隐私。  

潜在偏见，如：  
* 在数据收集过程中，仅选取大学生群体作为样本，而忽略其他重要的人群。

🤔基于这些数据的模型公正吗？  


**心理学研究背后的内隐哲学观**

在研究过程中未被明确提及，却深刻影响研究设计和解释的潜在观念和假设形成了心理学研究背后的内隐哲学观。

主要包括普遍性假设导致的对文化差异考量不足，以及资本主义核心观念导致的研究样本偏倚。这些内隐观念可能使研究忽视文化多样性和社会不平等，从而影响研究的全面性、准确性和应用价值。

|内隐哲学观方面|重要性|作用|影响|新的提倡|
|----|----|----|----|----|
|普遍性假设与文化差异考量不足|构建准确全面且具文化适应性理论|使研究忽略文化因素，影响各环节|限制理论普适性，阻碍心理学全球发展|数据收集多样化、文化敏感性培训、理论构建多元视角|
|资本主义核心观念与研究样本偏倚|确保研究样本多样性和代表性|导致样本选择偏向西方或资本主义文化个体|研究结果有偏差，不利于跨文化研究|扩大样本来源、提高研究者文化敏感性、融合多元文化理论|

> 参考文献：Bettache, K. (2024). Where Is Capitalism? Unmasking Its Hidden Role in Psychology. *Personality and Social Psychology Review*, *0*(0). https://doi.org/10.1177/10888683241287570


**2. 研究目的，以及数据收集的目的公平吗？**  

在本示例研究中，研究目的来源自心理学家的好奇和假设。这是合理的，因为心理学研究是一种探索性的研究方法。  

一些极端的反例：  
* 如果研究项目来源于开发缓解压力药的厂商。那么，研究目的就可能被操纵，以支持药厂的销售。  
* 例如，有目的的选择被试。  
* 例如，有目的性的将实验目告诉被试，从而收集到符合预期的数据。

**数据由谁收集，数据收集的目的是什么？** 
   
数据收集者的身份、意图和研究目的，都会影响数据的性质和使用方式。
  
研究者可能内隐地假设心理学现象具有普遍性，但未充分考虑文化差异，这种内隐假设影响了结果的解释，忽视了不同社会和文化背景对心理现象的深刻影响(Ghai, Forscher, & Chuan-Peng, 2024)。

从这张图中，你可以发现什么？🤔

![Image Name](https://cdn.kesci.com/upload/sno3jb9h3b.png?imageView2/0/w/720/h/960)

> 参考文献：Ghai, S., Forscher, P.S. & Chuan-Peng, H. Big-team science does not guarantee generalizability. *Nat Hum Behav* **8**, 1053–1056 (2024). https://doi.org/10.1038/s41562-024-01902-y



<div style="padding-bottom: 30px;"></div>

**3. 模型分析的结果，将对个人和社会产生什么影响？**  

在本示例研究中，研究结果(模型分析的结果)具有一定的理论意义和实际意义。  
* 在理论层面， 自我匹配范式中的模型能够帮助揭示个体如何处理与“自我”相关的信息。
* 在实际意义层面，通过这些模型，可以更精确地识别个体在自我认知过程中的偏差。 

**4. 分析过程中包含的偏见？**  

一些反例：  
* 假设在一次研究中使用多种问卷收集多种因变量，然后选择有相关性的变量进行报告?  
* 在多因素实验设计中，通过增加变量来获得显著的交互作用，并尝试多种简单效应分析。

在心理学研究中，模型公正性往往与***心理学研究的可重复性***相关。  

1. 数据的收集过程是怎样的？  

2. 数据由谁收集，数据收集的目的是什么？  

3. 数据收集的过程，以及分析的结果，将对个人和社会产生什么影响？  

4. 分析过程中可能会包含哪些偏见  
    * *p*-hacking/HARKing  

<div style="text-align: center;">
    <img src="https://pic2.zhimg.com/80/v2-778de6c621356bc2c23c3a09ff2b0be5_1440w.webp" 
         alt="Image Description" 
         style="width: 60%; height: auto;">
</div>

(来源：胡传鹏, ..., 彭凯平. (2016). 心理学研究中的可重复性问题:从危机到契机. *心理科学进展, 24*(9), 1504-1518. doi: 10.3724/SP.J.1042.2016.01504)

## 这个模型可能有多错误(How wrong is the model?)  


>  **<center>“all models are wrong, but some are useful.   ————George Box”</center>**  


* 尽管统计模型是对更复杂现实的简化表达，良好的统计模型仍然可以是有用的，并可以增进我们对世界复杂性的理解。  

* 因此，在评估模型时，要问的下一个问题不是模型是否错误(is the model wrong?)，而是模型错误的程度(How wrong is the model?)  

🤔思考贝叶斯线性回归模型的假设在多大程度上与现实相符？


### 模型假设的影响  

**🤔 我们知道模型存在前提预设(assumption)，如果这些模型的前提预设不成立，模型会变得多糟糕？**  

在lec9中，我们使用一个线性模型来定义标签“Label”与反应时间“RT”之间的关系，并且指定了该模型成立的一些前提预设。  

$$  
\begin{align*}  
Y_i &= \beta_0 + \beta_1 X_i + \epsilon \;\;\;\;\;\;\;\;\epsilon \sim N(0,\sigma^2)\\  
&\Downarrow \\  
Y_i | \beta_0, \beta_1, \sigma &\stackrel{ind}{\sim} N\left(\mu_i, \sigma^2\right) \;\; \text{ with } \;\; \mu_i = \beta_0 + \beta_1X_i  .\\  
\end{align*}  
$$  

* 回归模型需满足如下假设：  

    1. 独立观测假设: 每个观测值$Y_i$是相互独立的，即一个观测的值不受其他观测的影响.  

    2. 线性关系假设: 预测值$\mu_i$和自变量$X_i$之间可以用线性关系来描述，即：$\mu_i = \beta_0 + \beta_1 X_i$.  

    3. 方差同质性假设: 在任意自变量的取值下，观测值$Y_i$都会以$\mu_i$为中心，同样的标准差$\sigma$呈正态分布变化（$\sigma$ is consistent）.  



**1. 当假设1 (独立观测假设) 被违反时：**  

* 在心理学的实验数据中，观测值之间常常存在依赖关系 (dependent)。  
* 比如，反应时数据在单个被试内、或某种特定刺激类型内可能表现得更加同质：  
  * 有的被试（Participant）总是比其他人反应得更快；
  * 有的刺激类型（Stimulus）可能总是导致更快的反应。  
* 这种观测值的相互关联会导致对结果的不准确估计，具体体现在：
  - **过低的标准误**：错误高估了参数的显著性；
  - **错误的效应估计**：未能正确捕捉组间差异。  
* 解决这种关联问题需要采用层级模型（Hierarchical Models），特别是**层级贝叶斯模型（Hierarchical Bayesian Models）**。这种方法能够同时建模个体差异（如被试间的反应）和组间差异（如刺激类型的影响）。

例如，在下图中：
* 左图可能低估了被试间的变异性，假设所有被试的反应时间都完全由刺激难度解释。  
* 右图通过引入随机截距（Random Intercept）更好地捕捉了被试间的差异，使得模型更贴合数据的实际结构。

![Image Name](https://cdn.kesci.com/upload/s3xt2u50rw.png?imageView2/0/w/960/h/960)  
1. **左图：单一模型（无随机效应）**  
   - 仅考虑整体平均效应，没有控制“被试”或“刺激”的特定差异。  
   - 每个数据点的误差条（灰色线条）表示高水平的变异性，线性趋势可能无法很好地反映个体间差异。
2. **右图：考虑随机截距模型（Random Intercept Model）**  
   - 模型中引入了**被试间的随机截距**（by-participant random intercept），将不同被试的反应时间（RT）的基本差异纳入建模过程。  
   - 虚线代表各被试的随机截距，展示了“被试间”的系统性差异；实线代表整体效应估计（包含群体水平的趋势）。  
   - 结果更加细化，并能够同时反映群体趋势和个体变异。

> source: Brown, V. A. (2021). An Introduction to Linear Mixed-Effects Modeling in R. *Advances in Methods and Practices in Psychological Science, 4*(1), 2515245920960351. https://doi.org/10.1177/2515245920960351

<div style="padding-bottom: 30px;"></div>

**2. 当假设2(线性关系假设)和假设3(方差同质性假设)被违反时：**  

* 假设2(线性关系假设)违反的情况：在左图中，我们可以看到，Y和X之间的关系并非线性的  

* 假设3(方差同质性假设)违反的情况：并且，随着X的增大，Y的变异性越来越大。  

  * 这要导致的后果是，后验预测分布比实际观测值的分布差异很大(右图)  

![Image Name](https://www.bayesrulesbook.com/bookdown_files/figure-html/nonlinear-1-ch10-1.png)  



### 对模型进行修改  

* 考虑到并不是所有数据都会满足这些假设，当这些假设被违反时，我们需要考虑修改模型。

对于违反假设1 的情况，我们在之后会学习使用层级贝叶斯模型来处理相互关联的数据。  

**对于违反假设2和3的情况，通常有两种处理方式**  

**a. 使用不同的数据模型**  

* 不假设实际值与观测值之间的关系是正态的  $Y_i \sim N(\mu_i, \sigma^2)$  

* 之后，我们会学习使用其他回归模型来描述其数据关系，比如泊松回归、二项回归、负二项回归。  


**b. 对数据进行变换**  

> 如果数据模型并不是我们要担心的问题，我们可以对数据进行变换，仍然可以对变换后的数据可以使用正态模型：  

* 对$Y$进行变换：$g(Y_i) | \beta_0, \beta_1, \sigma \stackrel{ind}{\sim}N(\mu_i, \sigma^2)$，$\mu_i = \beta_0 + \beta_1 X_i$  

* 对$X$进行变换： $Y_i | \beta_0, \beta_1, \sigma \stackrel{ind}{\sim}N(\mu_i, \sigma^2)$，$\mu_i = \beta_0 + \beta_1 h(X_i)$  

* 同时对$Y$和$X$进行变换：$g(Y_i) | \beta_0, \beta_1, \sigma \stackrel{ind}{\sim}N(\mu_i, \sigma^2)$， $\mu_i = \beta_0 + \beta_1 h(X_i)$

在刚刚的例子当中，我们对$Y$的取值做一个对数变换  

$$  
\log(Y_i) | \beta_0, \beta_1, \sigma \stackrel{ind}{\sim}N(\mu_i, \sigma^2) \;\; \text{ with } \; \mu_i = \beta_0 + \beta_1 X_i  
$$  

* 在变换之后，可以看到$\log(Y)$与$X$之间的关系仍然是线性的，且随着$X$的增大，$Y$的变异性仍然是一致的。  

* 可以使用正态的线性模型来拟合$\log(Y)$的后验分布  
* 这部分内容将在逻辑回归(logistic regression)部分进行详细介绍，这里先不作深入解释。  

<table>  
        <tr>  
            <td><img src="https://www.bayesrulesbook.com/bookdown_files/figure-html/nonlinear-1-ch10-1.png" alt="" width="500" height="250"></td>  
            <td><img src="https://www.bayesrulesbook.com/bookdown_files/figure-html/nonlinear-2-ch10-1.png" alt="" width="500" height="250"></td>  
        </tr>  
        <tr>  
            <td>变换之前</td>  
            <td>log变换之后</td>  
        </tr>  
</table>  


## 模型评估  

一般情况下，我们的贝叶斯模型不会是完全不公平的，或者错得太离谱的。 但除了这些问题，**更为重要的是，模型是否可以用来准确预测新数据 Y 的结果。**  

> 如果说模型公平性和模型错误描述的是模型在**质量上**的优劣，那模型评估与比较就是在**数量上**衡量模型预测的准确性。  

我们可以通过什么指标来评估预测模型的整体质量呢？

- 绝对指标，衡量模型对于样本的预测能力。  
- 相对指标，衡量模型对于样本外数据的预测能力，也考虑了模型的复杂度。  

首先，我们先向大家介绍可用于评估模型在**样本**数据上的预测能力的绝对指标 ---- 绝对误差的中位数，median absolute error (MAE) 
- 衡量观测值和其后验预测均之间的典型差异。

### 绝对误差的中位数，median absolute error (MAE) 

**定义**：MAE 是观测值 $(Y_i)$ 和后验预测均值 $( Y_i')$ 的绝对误差的中位数，公式为：
$$ \text{MAE} = \text{median}(|Y_i - Y_i'|) $$

-  $(Y_i)$ ：实际观测值。
- $( Y_i')$：模型后验预测的均值。

- 假设 $Y_1, Y_2…, Y_n$表示n个观察结果。  

- 每个 $Y_i$ 都有对应的后验预测值，其均值为 $Y_i'$  
  
**作用**：衡量模型预测值和实际观测值之间的典型差异。

**特点**：
- MAE 是绝对误差的典型值，对异常值的敏感性较低。
- 作为绝对指标，直接反映模型的预测精度。


### 相对指标：样本外预测能力

仅在样本数据上评估模型并不足以验证其泛化能力，尤其是心理学数据经常受到时间和抽样偏差的影响。例如：

- 比如，个体的压力状态可能随着季节变化，因此在不同季节收集到的数据会受到时间的影响。  
- 抽样差异：训练模型的数据可能来自理工科学生，而测试模型的数据来自心理学学生，这种抽样差异可能导致预测性能下降。


因此，一种更高效的方法是，一次性多收集一些数据，选择其中的一部分作为预测数据。

而**相对指标**更多的是评估模型在**样本外**数据上的，同时考虑模型的复杂度,这更有利于比较不同模型的预测能力。

### 交叉验证(cross validation)  

但问题在于，我们选择哪一部分数据作为预测数据呐？或者说，我们该如何有效的对数据进行抽取呐？  

**交叉验证(cross validation)** 的目的就在于：提供不同的抽取预测数据的策略  
- 其关键在于从已有样本中拿出一部分数据当作预测数据。  


![](https://pic1.zhimg.com/80/v2-e9ad5ba61cda7ebd02848f336607eb70_1440w.webp)  


> 资料来源：【绝对干货】机器学习模型训练全流程！- 知乎 https://zhuanlan.zhihu.com/p/184673895

常见的交叉验证策略：  
1. 分半交叉验证 (Split-half cross-validation)  
	- 分半交叉验证将观测数据对半分成两部分，分别在不同的数据集上拟合模型，并在另外一半数据集上验证模型，最后再对比不同的模型在两份数据集作为验证集时的预测准确度。  
2. K 折交叉验证 (K-fold cross-validation)  
	- K 折交叉验证把数据分成 K 分，其中一份作为训练集（拟合模型，对参数进行估计），其余的 K-1 分数据集作为验证集，总共重复这个流程 K 次。以 K 次验证结果的均值作为验证标准。  
3. 留一法交叉验证 (Leave-one-out cross-validation)  
	- 留一法交叉验证是 K 折交叉验证的一个特例，当分折的数量等于数据的数量时，K 折留一法便成了留一法交叉验证。留一法交叉验证相较于普通的交叉验证方法，几乎使用了所有数据去训练模型，因此留一法交叉验证的训练模型时的**偏差 (bias) 更小、更鲁棒**，但是又因为验证集只有一个数据点，验证模型的时候**留一法交叉验证的方差 (Variance) 也会更大**。

**K 折交叉验证 (K-fold cross-validation)**  

K 折交叉验证在分半交叉验证的基础上，将数据集分成 K 份(称为 CV-K)，其中一份作为测试集，其余 K-1 份作为训练集，重复这个流程 K 次。  

K 折交叉验证，以 K 次测试结果的**均值**作为验证标准。例如，在压力-自我控制的例子中：  
- 我们可以使用 K=5 折的交叉验证，将数据集分成 5 份，每次使用 4 份数据作为训练集，1份数据作为测试集。  
- 对每一次迭代，我们使用 4 份数据训练模型，然后使用剩下的一份数据进行测试，并计算相应的MAE。  
- 重复这个流程 5 次，然后取每次MAE测试结果的均值作为最终的测试结果。  

![](https://pic3.zhimg.com/80/v2-ff846ee7eefdcd425e123d9d31b4d58a_1440w.webp)  

> 资料来源：【绝对干货】机器学习模型训练全流程！- 知乎 https://zhuanlan.zhihu.com/p/184673895

**留一法交叉验证 (Leave-one-out cross-validation)**  

留一法交叉验证是 K 折交叉验证的一个特例，当分折的数量K等于数据的数量n时，K 折留一法便成了留一法交叉验证。  
- 留一法交叉验证相较于普通的交叉验证方法，几乎使用了所有数据去训练模型。  
- 留一法交叉验证 (Leave-one-out cross-validation)的缩写为 loo-cv，或者 loo。  

![](https://www.baeldung.com/wp-content/uploads/sites/4/2022/05/loso.png)  

> 资料来源：https://www.baeldung.com/cs/cross-validation-k-fold-loo

### ELPD (Expected log-predictive density)  

留一法交叉验证 LOO (包括之前的交叉验证方法)是用于评估模型在**未知数据**上预测能力的思想框架，其本身并不提供具体的统计指标。  

**ELPD** (Expected log-predictive density) 是 LOO 方法的具体实现，以对数似然函数作为统计指标。  

其计算步骤：  
- 同 K 折交叉验证一样，首先将数据集分成 n 份，n为数据总的数量。  
- 利用 n-1 份数据去训练模型，得到后验模型 $p(\theta_{-i}|y_{-i})$。  
- 使用剩下的一份数据作为测试数据 $y_{i}$，计算后验预测模型 $p(y_{i}|y_{-i})$。  
- 重以上过程，重复 n 次，得到 n 个后验预测模型,并计算其对数化后的期望值 $E(log(p(y_{i}|y_{-i})))$。  


![Image Name](https://cdn.kesci.com/upload/s4bmgg7han.png?imageView2/0/w/640/h/640)  


### 补充：其他指标

之前讨论过模型评估中两种类型的指标：  
- 绝对指标，衡量模型对于样本的预测能力。  
- 相对指标，衡量模型对于样本外数据的预测能力，也考虑了模型的复杂度。  

这两种指标包含了多种具体的统计值：  

模型拟合优度的方法包括：  
- MAE or MSE(mean square error)  
- 对数似然 (log likelihood)  
- $R^2$  

模型预测进度的方法包括：  
- AIC  
- DIC  
- WAIC  
- LOO-CV  


|                    | AIC                                  | DIC                                      | WAIC       | LOOCV           | BIC                                  |  
| ------------------ | ------------------------------------ | ---------------------------------------- | ---------- | --------------- | ------------------------------------ |  
| 适用框架           | 频率论                               | 贝叶斯                                   | 贝叶斯     | 贝叶斯          | 贝叶斯/频率论                        |  
| 偏差（deviance）   | 最大似然参数 $\theta_mle$ 的对数似然 | 贝叶斯参数均值 $\bar{\theta}$ 的对数似然 | LPPD       | $ELPD_{LOO-CV}$ | 最大似然参数 $\theta_mle$ 的对数似然 |  
| 矫正（correction） | 参数数量                             | 似然的变异                               | 似然的变异 |     由于采用 LOO-CV 思想，因此不需要矫正            | 参数数量+数据数量                    |

目前认知建模在科学心理学中得到了广泛应用，而模型比较作为认知建模的核心环节，不仅是评估模型对数据的拟合优度（平衡过拟合与欠拟合），还需要考虑模型复杂度对预测能力的影响。

然而，由于模型比较指标种类繁多，研究者在选用时往往面临困惑。

在郭鸣谦等(2024) 的文章中便把常用指标划分为三类，其中AIC、DIC、WAIC 等基于交叉验证的指标，通过数据分割或后验分布计算预测性能，平衡拟合优度和复杂度。

![Image Name](https://cdn.kesci.com/upload/snng3h8fir.png?imageView2/0/w/640/h/640)

> 郭鸣谦, 潘晚坷, 胡传鹏. (2024). 认知建模中模型比较的方法. *心理科学进展*, *32*(10), 1736-1756. doi: 10.3724/SP.J.1042.2024.01736



## 偏差-方差权衡 (Bias-Variance Trade-off)

- 偏差（Bias）：指的是算法的期望预测与真实预测之间的偏差程度， 反应了模型本身的拟合能力。
  
- 方差（Variance）：用不同训练数据进行模型评估时，模型表现的变化程度。

在模型训练过程中，偏差和方差之间存在一定的权衡关系：

- 高偏差通常伴随着低方差，即模型较为简单，但能够在不同训练数据集上保持较为稳定的表现；
- 低偏差通常伴随着高方差，即模型较为复杂，可以在训练数据上做得很好，但在其他数据集上表现不稳定；
- 因此，偏差和方差之间需要达到一种平衡，即偏差-方差权衡，以避免模型过于简单或过于复杂。


在模型评估中，无论是**绝对评估**还是**相对评估**，都可以结合偏差-方差权衡的概念来理解。偏差-方差权衡揭示了以下关键事实：  

- **模型越复杂**：虽然能够更好地拟合训练数据，但往往会失去对样本外数据的解释能力（即过拟合）。  
- **模型越简单**：尽管能够在不同样本间保持一致性，但对于任何特定样本的解释力可能较弱（即欠拟合）。  

举例说明：
- 如果我们的目标是建立一个能够准确预测响应变量 \( Y \) 的模型，就需要包含足够多的预测因子，以获得对 \( Y \) 的充分信息。  
- 然而，加入过多的预测因子可能适得其反。模型不仅会过度拟合训练数据，还可能导致复杂性增加，从而降低泛化能力。

通过平衡模型的偏差和方差，我们可以选择一个既能够捕捉数据结构，又具有良好预测能力的模型。这种权衡是建模过程中不可忽视的核心问题。

![Image Name](https://vitalflux.com/wp-content/uploads/2020/12/overfitting-and-underfitting-wrt-model-error-vs-complexity-768x443.png)  

资料来源：https://vitalflux.com/overfitting-underfitting-concepts-interview-questions/

模型评估的核心在于模型捕捉到了数据中的关键模式，既非太简单而错过数据中有价值的信息(**欠拟合, underfitting**)，也不会太复杂从而将数据中的噪音加入到模型中(**过拟合, overfitting**)。  

**欠拟合(underfitting)**  

* 欠拟合的模型在当前样本的数据拟合效果不好，且其泛化能力(模型在当前样本外新的数据上的预测的准确度)也同样不佳。  
* 导致欠拟合的原因  
  * 数据特征较少  
    * 数据特征指的是数据的属性，比如第一部分中展示的数据的各个变量就是数据的特征。在所有变量都能独立地对目标变量做出解释的前提下，数据特征越多，数据拟合程度越好。  
  * 模型复杂度过低  
    * 模型的复杂度代表模型能够描述的所有函数，比如线性回归最多能表示所有的线性函数。  
    * 模型的复杂度和模型的参数数量有关，一般来说，模型参数越多，复杂度越高，模型参数越少，复杂度越低。  

**过拟合(overfitting)**  

* 模型在当前样本的数据上的拟合程度极好，但是泛化能力也较差。  
* 模型把训练样本学习地“太好了”，把样本自身地一些噪音也当作了所有潜在样本都会具有的一些性质，这样就会导致其泛化性能下降。  
* 导致过拟合的原因  
  * 当前样本的噪音过大，模型将噪音当作数据本身的特征  
  * 当数据的有些特征与目标变量无关，这些特征就是噪音，但它也可能被误当作数据特征，这就会造成模型过拟合  
  - 样本选取有误，样本不能代表整体  
  - 模型参数太多，模型复杂度太高  



![Image Name](https://cdn.kesci.com/upload/s4bp7piw04.png?imageView2/0/w/640/h/640)  


资料来源：https://blog.csdn.net/weixin_43378396/article/details/90707493

### 如何避免欠拟合  

- 增加数据的特征  

- 增加模型复杂度  

### 如何避免过拟合  

- 选择更具代表性的数据  

- 降低模型复杂度  


**问题的本质在于：模型与数据真实的生成模型匹配**  

为了选择一个能够在过拟合和欠拟合之间的达到平衡的最佳模型，就需要进行模型评估、比较和选择。


## 模型评估指标的代码演示

在我们了解模型评估的基本原理和方法后，接下来我们通过代码来演示如何使用这些方法来评估模型。包括：
- 绝对指标，MAE
- 相对指标，ELPD-LOO

### 计算 MAE

MAE 是观测值 $(Y_i)$ 和后验预测均值 $( Y_i')$ 的绝对误差的中位数，公式为：
$$ \text{MAE} = \text{median}(|Y_i - Y_i'|) $$

-  $(Y_i)$ ：实际观测值。
- $( Y_i')$：模型后验预测的均值。

接下来我们通过代码来演示如何计算 MAE，以 model 1 为例。


1. 提取后验预测数据，从模型中提取后验预测数据是后续计算的基础。

In [8]:
# 从采样结果中提取后验预测样本
posterior_predictive = model1_trace.posterior_predictive["Y_obs"]

# 在draw 和 chain 两个维度上计算后验均值
posterior_mean = posterior_predictive.mean(dim=["chain", "draw"])

posterior_mean.head()

<xarray.DataArray 'Y_obs' (Y_obs_dim_2: 5)>
array([0.70285717, 0.70416979, 0.76634711, 0.76563904, 0.76169855])
Coordinates:
  * Y_obs_dim_2  (Y_obs_dim_2) int32 0 1 2 3 4

2. 计算 MAE
通过提取的后验预测值，计算 MAE，作为模型预测误差的绝对指标。

In [9]:
# 计算 MAE（观测值和后验均值的绝对误差的中位数）
mae = np.median(np.abs(df["RT_sec"] - posterior_mean))

print(f"MAE: {mae}")

MAE: 0.08791264586944775


对于 MAE 的解读。  

- 绝对误差的中位数，median absolute error (MAE)衡量了预测观测值$Y_i$与后验预测均值之间$Y_i'$的差异。  
- **MAE越小**表明后验模型的**预测越准确**。  

我们可以对比三个模型的 MAE 结果。

In [10]:
def calculate_mae(trace, observed_data, dv = "Y_obs"):
    """
    计算后验预测均值和 MAE (Median Absolute Error)。
    
    Parameters:
    - trace: PyMC 模型的采样结果 (InferenceData 对象)。
    - observed_data: 包含真实观测值的 Pandas DataFrame。
    - dv: 需要计算 MAE 的数据列名，默认为 "Y_obs"。

    Returns:
    - posterior_mean: 后验预测值的均值。
    - mae: 后验预测均值与观测值之间的 MAE。
    """

    # 提取后验预测值
    posterior_predictive = trace.posterior_predictive[dv]
    
    # 计算后验预测均值（在 draw 和 chain 两个维度上取平均值）
    posterior_mean = posterior_predictive.mean(dim=["chain", "draw"])
    
    # 计算 MAE（绝对误差的中位数）
    mae = np.median(np.abs(observed_data - posterior_mean))
    
    return mae

In [11]:
pd.DataFrame({
    "Model 1": [calculate_mae(model1_trace, df["RT_sec"], "Y_obs")],
    "Model 2": [calculate_mae(model2_trace, df["RT_sec"], "Y_obs")],
    "Model 3": [calculate_mae(model3_trace, df["RT_sec"], "Y_obs")],
})

,Model 1,Model 2,Model 3
0,0.087913,0.088987,0.089136


### 计算 ELPD-LOO

在实际操作中，我们通过 `ArViz` 的函数`az.loo`计算 $ELPD_{LOO-CV}$。  

* 在`az.loo`返回的值中，`elpd_loo` 为$E(log(p(y_{i}|y_{-i})))$，  
* `elpd_loo`越高表示模型的预测值越精确  

注意：由于 $ELPD_{LOO-CV}$ 的计算量也比较大，ArViz 会使用 Pareto Smooth Importance Sampling Leave Once Out Cross Validation (PSIS-LOO-CV) 来近似 $ELPD_{LOO-CV}$。  

PSIS-LOO-CV 有两大优势：  
1. 计算速度快，且结果稳健  
2. 提供了丰富的模型诊断指标  

注意：  
- 要计算 elpd_loo 需要在采样 `pm.sample` 中加入 `idata_kwargs={"log_likelihood": True}`  
- 或者，在模型采样完成后计算对数似然，即 `with model:  pm.compute_log_likelihood(model_trace)`。

首先，我们以 model 3为例

In [ ]:
# 以 model 3 为例计算elpd_loo

az.loo(model3_trace)

Computed from 20000 posterior samples and 186 observations log-likelihood matrix.

         Estimate       SE
elpd_loo   107.87    10.24
p_loo        6.90        -
------

Pareto k diagnostic values:
                         Count   Pct.
(-Inf, 0.70]   (good)      186  100.0%
   (0.70, 1]   (bad)         0    0.0%
   (1, Inf)   (very bad)    0    0.0%

模型`elpd_loo`的结果为107.87

- 然而，仅凭单个值，并不能反映模型的预测精确程度。  
- 虽然 **ELPDs 无法为任何单一模型的后验预测准确性提供可解释的度量，但它在比较多个模型的后验预测准确性时非常有用**。  

我们可以通过 `arviz.compare` 方法来对比多个模型的 elpd。从下面结果可见：  
- 模型1的 elpd_loo 最大，表明它对**样本外数据**的预测性能最好。  
- 而模型3的 elpd_loo 最小，表明它的预测性能最差。  
- 并且这些结果与我们通过 MAE 计算及上节课的贝叶斯因子计算和HDI+rope 区间计算得到的判断一致。  

需要注意的是：  
- arviz 提供的结果包括了 elpd se，这使得我们可以判断两个模型的预测差异 elpd_diff 是否超过两至三个标准误se。  

In [48]:
comparison_list = {
    "model1":model1_trace,
    "model2":model2_trace,
    "model3":model3_trace,
}
az.compare(comparison_list)

,rank,elpd_loo,p_loo,elpd_diff,weight,se,dse,warning,scale
model1,0,109.551050,4.073800,0.000000,8.388306e-01,10.125275,0.000000,False,log
model2,1,109.143428,4.994407,0.407622,1.611694e-01,9.949396,1.098084,False,log
model3,2,107.874296,6.901146,1.676754,1.110223e-16,10.235348,1.516092,False,log


### 补充：DIC在Python中的实现

在贝叶斯统计中，DIC 适用于基于 MCMC (Markov chain Monte Carlo) 采样估计的模型。

由于PyMc中没有直接计算DIC的方式，我们补充这一内容，方便同学们在练习中参考。


**DIC 的计算公式为：**

$$
\text{DIC} = -2D(\bar{\theta}) + 2 \times p_D
$$

- 其中，$\bar{\theta}$ 为参数后验分布的均值，而 $D(\theta)$ 则是真实数据与模型预测分布之间的偏差（Deviance），用以衡量模型的性能；

- DIC 公式的第一项是 $-2$ 乘上参数后验分布上的均值的偏差，代表了模型拟合的程度；

- 第二项 $p_D$ 被称作有效参数（effective number of parameters），是模型拟合的复杂度的惩罚项。

DIC 综合考虑了模型的拟合优度和复杂度。模型的 DIC 值越低，说明模型在平衡拟合优度与复杂度后表现越好。

DIC（Deviance Information Criterion）是用于模型选择和评估的指标，通常用于贝叶斯模型。在统计学中，DIC 是一种衡量模型拟合优度和复杂度的指标，类似于AIC（Akaike Information Criterion）和BIC（Bayesian Information Criterion）。

**偏差的公式为：**

$$
D(\theta_s) = \log L(y|\theta_s) \tag{15}
$$

其中 $s$ 代表了 MCMC 的样本，因此 $\theta_s$ 是 MCMC 样本的参数值。

**有效参数 $p_D$**

$p_D$ 为有效参数(effective number of parameters), 是模型拟合的复杂度的惩罚项, 计算公式如下：
$$
p_D = \text{Var}(D(\theta)) = \frac{1}{M} \sum_{m=1}^{M} D(\theta^{(m)}) - D(\hat{\theta})
$$
- 其中，$D(\theta^{(m)})$ 是第 $m$ 个样本的偏差，$\hat{\theta}$ 是后验均值。

**代码实现：**

要根据模型的 **log-likelihood** 结果计算 DIC (Deviance Information Criterion)，可以按照以下步骤操作：

假设 `model_trace.log_likelihood["Y_obs"]` 是后验采样的 log-likelihood 值。

**log-likelihood 的结构**：
- `log_likelihood` 是一个包含多个链和采样的对数似然值矩阵。
- 在 `chain` 和 `draw` 维度上计算均值可以得到每个观测值的平均 log-likelihood。

首先，我们以model1为例进行演示

In [24]:
def calculate_dic(log_likelihood):
    """
    根据 log-likelihood 计算 DIC (Deviance Information Criterion)。 参考 Evans, N. J. (2019). Assessing the practical differences between model selection methods in inferences about choice response time tasks. Psychonomic Bulletin & Review, 26(4), 1070–1098. https://doi.org/10.3758/s13423-018-01563-9
    
    Parameters:
    - log_likelihood: xarray 数据集，包含每个链和样本的 log-likelihood 值。
    
    Returns:
    - dic: 计算得到的 DIC 值。
    """
    # 计算每个样本的Deviance
    deviance_samples = -2 * log_likelihood
    
    # 计算平均Deviance
    D_bar = deviance_samples.mean()
    
    # 计算有效自由度 p_D
    p_D = deviance_samples.max() - D_bar
    
    # 计算DIC
    DIC = -2 * (D_bar - p_D)
    
    return DIC["Y_obs"].values

In [25]:
# 调用 compute_dic 函数
dic_value = calculate_dic(model1_trace.log_likelihood)
print(f"DIC 值: {dic_value}")

DIC 值: 28.29209986121615


同样我们可以计算所有模型（model1，model2，model3）的 DIC 值进行对比：

In [26]:
pd.DataFrame({
    "Model 1": [calculate_dic(model1_trace.log_likelihood)],
    "Model 2": [calculate_dic(model2_trace.log_likelihood)],
    "Model 3": [calculate_dic(model3_trace.log_likelihood)],
})

,Model 1,Model 2,Model 3
0,28.29209986121615,24.679373842803408,27.387719057819087


🤔思考：  

1. 如果你的目标是在不控制任何其他因素的情况下探索反应时间和什么有关，你会使用哪种模型？  
    
    - 考虑到模型1优于模型2和模型3--因此，选择模型1可能能更好地反应反应时间的变化。 
        
2. 如果你的目标是最大限度地提高模型的预测能力，而在模型中只能选择一个预测因子，您会选择Label还是Matching？  
    
    - 由于模型1优于模型2，如果仅选择一个预测变量的话，选择Label能获得对于反应时间更好的预测。  
        
3. 这四个模型中，哪个模型的**总体预测结果**最好？  
    - 模型1以微弱优势超过了使用所有预测因子的模型2。这表明，在建立模型的过程中，预测因子并不是越多越好。  
    - 事实上，模型3比模型2还差一点，这表明Label和matching之间的交互效应很弱，加入两者的交互项，会减弱模型的预测能力。  
 
 因此，为了简单高效，我们更有理由选择模型1。  
 
 ![](https://th.bing.com/th/id/R.31c49d0bc73e477eda2cf52fef1b859f?rik=fdM9GvPWnNMg4Q&riu=http%3a%2f%2fwww.esafety.cn%2fblog%2fUploadFiles%2f2018-7%2f51155444445.jpg&ehk=O2MXHgnsmdvS68QMhw6CaIw82aHg2E0q%2fSKKtutAZjk%3d&risl=&pid=ImgRaw&r=0)  
 
 > 资料来源: http://www.esafety.cn/Blog/u/9490/archives/2018/154367.html

## 练习: 当自变量为连续变量

### 模型回顾  

在第十课的练习部分，我们探究了自我控制水平是否压力和吸烟有关，分别建立了三个回归模型，本节课的练习将基于上节课建立的三个模型进行。

💡 如果上节课的练习没有完成，是无法完成本节课的练习的哦！（难度 🔝🔝🔝）


<table>  
    <tr>  
        <td>模型</td>  
        <td>model1</td>  
        <td>model2</td> 
        <td>model3</td>  
    </tr>  
    <tr>  
        <td>自变量</td>  
        <td>压力(连续变量)</td>  
        <td>压力(连续变量)，吸烟(离散变量)【无交互】</td> 
        <td>压力(连续变量)，吸烟(离散变量)【有交互】</td>   
    </tr>  
    <tr>  
        <td>自变量含义</td>  
        <td colspan="3">压力（14-70的压力评级）；吸烟（`0` 表示不吸烟，`1` 表示吸烟）</td>  
    </tr>  
    <tr>  
        <td>先验</td>  
        <td>  
            β0 ~ N(50, 10) <br>  
            β1 ~ N(0, 10) <br>  
            σ ~ Exp(0.6) <br>  
        </td>  
        <td>  
            β0 ~ N(50, 10) <br>  
            β1 ~ N(0, 10) <br> 
            β2 ~ N(0, 10) <br>   
            σ ~ Exp(0.6) <br>  
        </td> 
        <td>  
            β0 ~ N(50, 10) <br>  
            β1 ~ N(0, 10) <br> 
            β2 ~ N(0, 10) <br>  
            β3 ~ N(0, 10) <br>  
            σ ~ Exp(0.6) <br>  
        </td>  
    </tr>  
</table>



In [16]:
# 导入 pymc 模型包，和 arviz 等分析工具 
import pymc as pm
import arviz as az
import seaborn as sns
import scipy.stats as st
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import os

# 忽略不必要的警告
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# 通过 pd.read_csv 加载数据 Data_Sum_HPP_Multi_Site_Share.csv
try:
  df_re = pd.read_csv('/home/mw/input/bayes3797/Data_Sum_HPP_Multi_Site_Share.csv')
except:
  df_re = pd.read_csv('data/Data_Sum_HPP_Multi_Site_Share.csv')


# 筛选站点为"Tsinghua"的数据
df = df_re[df_re["Site"] == "Tsinghua"]

df = df[["stress","scontrol","smoke"]]

#1 表示吸烟，2表示不吸烟
df["smoke"] =  np.where(df['smoke'] == 2, 0, 1)
df["smoke_recode"] =  np.where(df['smoke'] == 1, "yes", "no")


#设置索引
df["index"] = range(len(df))
df = df.set_index("index")

In [ ]:
##-------------------------------------------------
#     定义模型4、5、6，补全...部分
#---------------------------------------------------


with pm.Model() as model4:

    beta_0 = pm.Normal("beta_0", mu=..., sigma=...)            
    beta_1 = pm.Normal("beta_1", mu=..., sigma=...)        
    sigma = pm.Exponential("sigma", ...)               

    x = pm.MutableData("smoke",df.stress)                   
    mu = pm.Deterministic("mu", beta_0 + beta_1*x)          

    likelihood = pm.Normal("y_est", mu=mu, sigma=sigma, observed=df.scontrol)


with pm.Model() as model5:

    beta_0 = pm.Normal("beta_0", mu=..., sigma=...)                
    beta_1 = pm.Normal("beta_1", mu=..., sigma=...)          
    beta_2 = pm.Normal("beta_2", mu=..., sigma=...)           
    sigma = pm.Exponential("sigma", ...)                   

    stress = pm.MutableData("stress",df.stress)                          
    smoke = pm.MutableData("smoke",df.smoke)                              
    mu = pm.Deterministic("mu", beta_0 + beta_1*stress + beta_2*smoke)    

    likelihood = pm.Normal("y_est", mu=mu, sigma=sigma, observed=df.scontrol) 

with pm.Model() as model6:
    beta_0 = pm.Normal("beta_0", mu=..., sigma=...)        
    beta_1 = pm.Normal("beta_1", mu=..., sigma=...)        
    beta_2 = pm.Normal("beta_2", mu=..., sigma=...)        
    beta_3 = pm.Normal("beta_3", mu=..., sigma=...)          
    sigma = pm.Exponential("sigma", ...)                 

    stress = pm.MutableData("stress",df.stress)      
    smoke = pm.MutableData("smoke",df.smoke)         
    mu = pm.Deterministic("mu", beta_0 + 
                                beta_1*stress + 
                                beta_2*smoke +
                                beta_3*stress*smoke)      

    likelihood = pm.Normal("y_est", mu=mu, sigma=sigma, observed=df.scontrol) 

In [ ]:
#========================================
#     注意！！！以下代码可能需要运行 5 分钟左右,直接运行即可
#     直接运行即可，无需修改
#========================================

def run_model_sampling(save_name, model=None, draws=2000, tune=1000, chains=4, random_seed=84735):
    """
    运行模型采样，并在结果不存在时进行采样，存在时直接加载结果。

    Parameters:
    - save_name: 用于保存或加载结果的文件名（无扩展名）
    - model: pymc 模型
    - draws: 采样次数 (默认5000)
    - tune: 调整采样策略的次数 (默认1000)
    - chains: 链数 (默认4)
    - random_seed: 随机种子 (默认84735)

    Returns:
    - trace: 采样结果
    """
    
    # 检查是否存在保存的.nc文件
    nc_file = f"{save_name}.nc"
    if os.path.exists(nc_file):
        print(f"加载现有的采样结果：{nc_file}")
        # 如果文件存在，则加载采样结果
        trace = az.from_netcdf(nc_file)
    else:

        assert model is not None, "模型未定义，请先定义模型"

        print(f"没有找到现有的采样结果，正在执行采样：{save_name}")
        # 如果文件不存在，则进行采样计算
        with model:
            trace = pm.sample_prior_predictive(draws=draws, random_seed=random_seed)
            idata = pm.sample(draws=draws,                   # 使用mcmc方法进行采样，draws为采样次数
                              tune=tune,                    # tune为调整采样策略的次数
                              chains=chains,                # 链数
                              discard_tuned_samples=True,   # tune的结果将在采样结束后被丢弃
                              idata_kwargs={"log_likelihood": True},
                              random_seed=random_seed)      # 后验采样

            trace.extend(idata)
            # 进行后验预测并扩展推断数据
            pm.sample_posterior_predictive(trace, extend_inferencedata=True, random_seed=random_seed)
            
            # 保存结果到指定文件
        trace.to_netcdf(nc_file)
        
    return trace


# 运行所有三个模型
model4_trace = run_model_sampling("lec10_model4",model4)
model5_trace = run_model_sampling("lec10_model5",model5)
model6_trace = run_model_sampling("lec10_model6",model6)

In [ ]:
# 直接运行即可，无需修改
# 将3个模型中的inference data 中的 y_est 统一改为 Y_obs

model4_trace = model4_trace.rename({"y_est": "Y_obs"})
model5_trace = model5_trace.rename({"y_est": "Y_obs"})
model6_trace = model6_trace.rename({"y_est": "Y_obs"})

### 计算MAE：  


In [ ]:
# 直接运行即可，无需修改
def calculate_mae(trace, observed_data, dv = "Y_obs"):
    """
    计算后验预测均值和 MAE (Median Absolute Error)。
    
    Parameters:
    - trace: PyMC 模型的采样结果 (InferenceData 对象)。
    - observed_data: 包含真实观测值的 Pandas DataFrame。
    - dv: 需要计算 MAE 的数据列名，默认为 "Y_obs"。

    Returns:
    - posterior_mean: 后验预测值的均值。
    - mae: 后验预测均值与观测值之间的 MAE。
    """

    # 提取后验预测值
    posterior_predictive = trace.posterior_predictive[dv]
    
    # 计算后验预测均值（在 draw 和 chain 两个维度上取平均值）
    posterior_mean = posterior_predictive.mean(dim=["chain", "draw"])
    
    # 计算 MAE（绝对误差的中位数）
    mae = np.median(np.abs(observed_data - posterior_mean))
    
    return mae

In [ ]:
##================================================
#                练习，修改... 部分
#                
#================================================

pd.DataFrame({
    "Model 4": [calculate_mae(model4_trace, df["..."], "...")],
    "Model 5": [calculate_mae(model5_trace, df["..."], "...")],
    "Model 6": [calculate_mae(model6_trace, df["..."], "...")],
})

,Model 4,Model 5,Model 6
0,3.417442,3.525749,3.548162


### 计算elpd_loo 

In [ ]:
##================================================
#                练习，修改... 部分       
#================================================

comparison_list = {
    "model4(contiunous)":...,
    "model5(multivariate )":...,
    "model6(interaction)":...,
}
az.compare(comparison_list)

### 计算DIC

In [ ]:
# 直接运行即可，无需修改
def calculate_dic(log_likelihood):
    """
    根据 log-likelihood 计算 DIC (Deviance Information Criterion)。 参考 Evans, N. J. (2019). Assessing the practical differences between model selection methods in inferences about choice response time tasks. Psychonomic Bulletin & Review, 26(4), 1070–1098. https://doi.org/10.3758/s13423-018-01563-9
    
    Parameters:
    - log_likelihood: xarray 数据集，包含每个链和样本的 log-likelihood 值。
    
    Returns:
    - dic: 计算得到的 DIC 值。
    """
    # 计算每个样本的Deviance
    deviance_samples = -2 * log_likelihood
    
    # 计算平均Deviance
    D_bar = deviance_samples.mean()
    
    # 计算有效自由度 p_D
    p_D = deviance_samples.max() - D_bar
    
    # 计算DIC
    DIC = -2 * (D_bar - p_D)
    
    return DIC["Y_obs"].values

In [ ]:
##================================================
#                练习，修改... 部分
#                
#================================================

DIC_list = {
    "m4_dic_value":calculate_dic(...),
    "m5_dic_value":calculate_dic(...),
    "m6_dic_value":calculate_dic(...),
}

## 总结  

本节课从不同模型评估的角度，介绍了模型评估与比较的基本思想。

通过学习，我们对贝叶斯分析的整体流程（Bayesian workflow）有了初步的理解。

在接下来的课程中，我们将不断实践这一流程，帮助大家更深入地领略贝叶斯分析的独特魅力。

![Image Name](https://cdn.kesci.com/upload/image/rkz1ehen1l.png?imageView2/0/w/960/h/960)

